<a href="https://colab.research.google.com/github/cheon4050/Ai-DeepLearning/blob/main/CIFAR10_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt

from torch.optim import lr_scheduler

batch_size=16
learning_rate=0.001
num_epoch=100


In [ ]:
cifar_train=dset.CIFAR10("CIFAR10/", train=True, transform=transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
]), target_transform=None, download=True)+dset.CIFAR10("CIFAR10/",train=True,transform=transforms.Compose([
                                      transforms.Resize((32,32)),
                                      transforms.RandomCrop(32, padding=4),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(), 
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) 
                                      ]),target_transform=None,download=True)
#+dset.CIFAR10("CIFAR10/", train=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5,0.5,0.5), std = (0.5,0.5,0.5))]), target_transform=None, download=True)
cifar_test=dset.CIFAR10("CIFAR10/", train=False, transform=transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
]),target_transform=None, download=True)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
def ComputeAccr(dloader, imodel):
  correct = 0
  total = 0
  
  for j, [imgs, labels] in enumerate(dloader):
    img = Variable(imgs,volatile=True).cuda()
    label = Variable(labels).cuda()
    output = imodel.forward(img)
    _, output_index = torch.max(output,1)

    total += label.size(0)
    correct += (output_index == label).sum().float()
  print("Accuracy of Test Data: {}".format(100*correct/total))

In [ ]:
train_loader = torch.utils.data.DataLoader(list(cifar_train)[:], batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = torch.utils.data.DataLoader(cifar_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)
class CNN(nn.Module):
  def __init__(self):
    super(CNN,self).__init__()
    self.layer = nn.Sequential(
        nn.Conv2d(3,16,3,padding=1),
        nn.BatchNorm2d(16),
        nn.LeakyReLU(),

        nn.Conv2d(16,32,3,padding=1),
        nn.BatchNorm2d(32),
        nn.LeakyReLU(),
        nn.MaxPool2d(2,2),

        nn.Conv2d(32,64,3,padding=1),
        nn.BatchNorm2d(64),
        nn.LeakyReLU(),
        nn.MaxPool2d(2,2)
    )
    self.fc_layer=nn.Sequential(
        nn.Linear(64*8*8,100),
        nn.BatchNorm1d(100),
        nn.ReLU(),
        nn.Dropout(0.6),
        nn.Linear(100,10),
        nn.LogSoftmax(dim=1)
    )
  def forward(self,x):
    out=self.layer(x)
    out=out.view(batch_size,-1)
    out=self.fc_layer(out)

    return out
model=CNN().cuda()

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
for i in range(1):
  model.train()
  for j, [image,label] in enumerate(train_loader):
    x = Variable(image).cuda()
    y_=Variable(label).cuda()
    optimizer.zero_grad()
    output=model.forward(x)
    loss=loss_func(output,y_)
    loss.backward()
    optimizer.step()

    if j%1000==0:
      print(j,loss)
  print(i)  
  model.eval()
  ComputeAccr(train_loader,model)
  ComputeAccr(test_loader,model)

0 tensor(0.4963, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(1.3288, device='cuda:0', grad_fn=<NllLossBackward>)
2000 tensor(0.6605, device='cuda:0', grad_fn=<NllLossBackward>)
3000 tensor(0.7485, device='cuda:0', grad_fn=<NllLossBackward>)
4000 tensor(0.5205, device='cuda:0', grad_fn=<NllLossBackward>)
5000 tensor(0.9359, device='cuda:0', grad_fn=<NllLossBackward>)
6000 tensor(0.4128, device='cuda:0', grad_fn=<NllLossBackward>)
0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 90.43099975585938
Accuracy of Test Data: 80.29999542236328


In [ ]:
model.eval()
ComputeAccr(test_loader,model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 80.29999542236328


In [ ]:
netname = './my_net01.pkl'
torch.save(model,netname)